In [13]:
#instantiate cursor and load libraries
import json
import pandas as pd
import requests


In [22]:
#create database
from mysql.connector import errorcode
db_name = 'Movies'

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [14]:
#specify database for connection
db = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = "Movies")

cursor = db.cursor()

In [15]:
import requests
from bs4 import BeautifulSoup as BS

In [46]:
#first page

page1 = requests.get('https://www.imdb.com/search/title/?title_type=feature&year=2018-01-01,2018-12-31&sort=boxoffice_gross_us,desc')
soup1 = BS(page.content, 'html.parser')
soup1.prettify()
info1 = soup.find_all(id = 'main')
movie1 = info[0].find_all(class_='lister-item mode-advanced')
movie1

[<div class="lister-item mode-advanced">
 <div class="lister-top-right">
 <div class="ribbonize" data-caller="filmosearch" data-tconst="tt5523010"></div>
 </div>
 <div class="lister-item-image float-left">
 <a href="/title/tt5523010/"> <img alt="The Nutcracker and the Four Realms" class="loadlate" data-tconst="tt5523010" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BOTg1NDI1MjEyMl5BMl5BanBnXkFtZTgwOTA5MTgwNjM@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB470041630_.png" width="67"/>
 </a> </div>
 <div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">51.</span>
 <a href="/title/tt5523010/">The Nutcracker and the Four Realms</a>
 <span class="lister-item-year text-muted unbold">(2018)</span>
 </h3>
 <p class="text-muted">
 <span class="certificate">PG</span>
 <span class="ghost">|</span>
 <span class="runtime">99 min</span>
 <span cl

In [35]:
 def get_score(movie):
        metascores = []
        for score in movie:
            metascore1 = score.find(class_="metascore").get_text()
            if metascore1 == 'None':
                metascore = 'None'
            else:
                metascore = metascore1
            metascores.append(int(metascore[:2]))
        return metascores

In [34]:
def find_fk(movie):
    ids = []
    for m in movie:
        id = m.find (class_="starBarWidget")['id']
        ids.append(id)
    return ids


In [33]:
def get_titles(movie):

    titles = []
    for one in movie:
        title = one.find('img')['alt']
        titles.append(title) 
    return titles



In [32]:
def get_stars(movie):
    stars = []
    for one in movie:
        x = one.find(class_='inline-block ratings-imdb-rating')
        stars.append(float(x['data-value']))
    return stars



In [31]:
def get_year(movie):
    
    years = []
    for one in movie:
        year = one.find(class_="lister-item-year text-muted unbold").get_text()
        parsed_year = year[-5:-1]
        years.append(int(parsed_year))
    return years


In [30]:
def get_gross(movie):
    gross_list = []
    for m in movie:
        gross = m.find(class_='sort-num_votes-visible')
        parsed_gross = gross.get_text(strip=True)
        parsed_gross2 = str.split(parsed_gross, sep='|')
       # print(parsed_gross2)
        parsed_gross3 = str.split(parsed_gross2[1], sep=':')
        parsed_gross4 = str.split(parsed_gross3[1], sep='.')[0]
        final_gross = str.split(parsed_gross4, sep='$')[1]
        gross_list.append(int(final_gross))
    return gross_list


In [70]:
#create table
DB_NAME = 'Movies'

create_query = """
      CREATE TABLE imdb_movies
    (
      imdb_id varchar(24),
      title varchar(50),
      year INT(4),
      star_rating FLOAT(3),
      metascore INT(5),
      gross INT(5)
      )
    """



In [47]:
var = 51
while var < 500:
    page = requests.get(f"https://www.imdb.com/search/title/?title_type=feature&year=2018-01-01,2018-12-31&sort=boxoffice_gross_us,desc&start={var}&ref_=adv_nxt")
    soup = BS(page.content, 'html.parser')
    soup.prettify()
    info = soup.find_all(id = 'main')
    movie = info[0].find_all(class_='lister-item mode-advanced')
    movie_df = make_df(movie)
    insert_movies(movie_df)

    var +=50


InterfaceError: Failed executing the operation; 'NoneType' object has no attribute 'convert_to_mysql'

In [44]:
#make into DF
def make_df(movie):
    all_movies_dataframe = pd.DataFrame({
        "imdb_id": find_fk(movie),
        "title": get_titles(movie),
        "year": get_year(movie),
        "star_rating": get_stars(movie),
        "metascore": get_score(movie),
        "gross(in millions)": get_gross(movie)
    })
    return all_movies_dataframe


In [49]:
#uses DF to insert into mySQL
def insert_movies(dataframe):
    
    all_movies_tuples = list(dataframe.itertuples(index=False, name=None))
    
    insert_statement = """INSERT INTO imdb_movies (imdb_id, title, year, star_rating, metascore, gross) 
            VALUES (%s, %s, %s, %s, %s, %s);"""
    cursor.executemany(insert_statement, all_movies_tuples)
    db.commit()
    db.close()


In [50]:
insert_movies(make_df(movie1))


InterfaceError: Failed executing the operation; 'NoneType' object has no attribute 'convert_to_mysql'